In [8]:
# ================================================================
#  3×3  H(α) = M₀ + α · diag(m)   –  fast interactive explorer
# ================================================================
import numpy as np, plotly.graph_objects as go, ipywidgets as W
from plotly.subplots import make_subplots
from IPython.display import display
import sympy as sp

# -----------------------------------------------------------------
#  α-grid (for the pre-computed tracks)
# -----------------------------------------------------------------
α_grid = np.linspace(-5, 5, 801)          # 0.01 spacing
nα     = len(α_grid)

# -----------------------------------------------------------------
#  default parameters  (dict; names used for sliders)
# -----------------------------------------------------------------
pars = dict(
    m11=0.0, m22=1.0, m33=2.0,                 # diagonal of M₁
    M0_12=1.0, M0_13=1.0, M0_23=1.0,           # upper-triangular M₀
    M0_11=0.0, M0_22=1.0, M0_33=2.0            # diagonal of M₀
)

# -----------------------------------------------------------------
#  helpers: matrices, eigen-tracks, asymptotes, polynomials
# -----------------------------------------------------------------
_super = str.maketrans('0123456789-','⁰¹²³⁴⁵⁶⁷⁸⁹⁻')     # Unicode superscripts
def cstr(x):                     # instead of s(x)
    return f"{x:+.3g}"

def fmt_poly(expr):
    """Return human-readable string for p(λ,α) with nice exponents."""
    poly = sp.Poly(expr, λsym, αsym)          # organise by (λ,α) powers
    terms = []
    for (i,j), c in sorted(poly.terms(), key=lambda t: (-sum(t[0]), t[0])):
        if abs(c) < 1e-12:
            continue
        t = cstr(c)              # <<<<<<<<<<
        if j: t += f" α{'' if j==1 else str(j).translate(_super)}"
        if i: t += f" λ{'' if i==1 else str(i).translate(_super)}"
        terms.append(t)
    out = " ".join(terms)
    return out.lstrip('+').replace('+ -', ' - ')          # tidy sign spacing

def M1(p):            # diag(m)
    return np.diag([p['m11'], p['m22'], p['m33']])

def M0(p):            # symmetric 3×3
    return np.array([[p['M0_11'], p['M0_12'], p['M0_13']],
                     [p['M0_12'], p['M0_22'], p['M0_23']],
                     [p['M0_13'], p['M0_23'], p['M0_33']]], float)

def eigen_tracks(p):
    M0_, M1_ = M0(p), M1(p)
    return np.array([np.linalg.eigvalsh(M0_ + a*M1_) for a in α_grid])

def asymptotes(p):          # just the diag of M₁, sorted
    return np.sort(np.diag(M1(p)))

# --- symbolic polynomial once per matrix change -----------------
αsym, λsym = sp.symbols('α λ')
def sym_poly_expr(p):
    H = sp.Matrix(M0(p)) + αsym*sp.Matrix(M1(p))
    return sp.expand(H.charpoly(λsym).as_expr())

# -----------------------------------------------------------------
#  first heavy step
# -----------------------------------------------------------------
tracks = eigen_tracks(pars)
Λ      = asymptotes(pars)
poly_sym_tex = sp.latex(sym_poly_expr(pars), order='lex')

# -----------------------------------------------------------------
#  Plotly figure: two stacked panels
# -----------------------------------------------------------------
fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.55, 0.45], vertical_spacing=0.10,
    subplot_titles=("Eigen-value tracks  λᵢ(α)",
                    "Characteristic polynomial  pₐ(λ)")
)
fig = go.FigureWidget(fig)

eig_cols = ['#1f77b4', '#ff7f0e', '#00cc96']
dash_col = 'rgba(120,120,120,0.6)'
WIN      = [-15, 15]

# -- eigen tracks (solid) & asymptotes (grey dashed)
for j,c in enumerate(eig_cols):
    fig.add_scatter(x=α_grid, y=tracks[:,j],
                    line=dict(color=c, width=3),
                    name=f'λ{j+1}', legendgroup='λ',
                    row=1, col=1)
for j in range(3):
    fig.add_scatter(x=α_grid, y=α_grid * Λ[j],
                    line=dict(color=dash_col, dash='dash'),
                    showlegend=False, row=1, col=1)

# -- vertical α-bar
fig.add_scatter(x=[0,0], y=WIN, mode='lines',
                line_color='rgba(255,0,0,0.35)', line_width=4,
                showlegend=False, row=1, col=1)

# -- black cubic & three root dots
λ_plot = np.linspace(-20, 20, 1601)
fig.add_scatter(x=λ_plot, y=np.zeros_like(λ_plot),
                line=dict(color='black'), showlegend=False,
                row=2, col=1)
for c in eig_cols:
    fig.add_scatter(x=[0], y=[0], mode='markers',
                    marker=dict(size=10, color=c),
                    showlegend=False, row=2, col=1)

# -- axes & layout
fig.update_xaxes(range=[-5,5],  title='α',   row=1,col=1)
fig.update_yaxes(range=WIN,     title='λ',   row=1,col=1)
fig.update_xaxes(range=[-20,20],title='λ',   row=2,col=1)
fig.update_yaxes(range=[-50,50],title='pₐ(λ)',row=2,col=1)

fig.update_layout(
    width = 700, height = 700,
    margin=dict(l=6,r=6,t=60,b=6),
    legend=dict(orientation='h', x=1.02, y=1.08, xanchor='right'),
    font=dict(size=13)
)

# -----------------------------------------------------------------
#  widgets
# -----------------------------------------------------------------
slider_cfg = dict(
    min=-20, max=20, step=0.01, continuous_update=True,
    layout=W.Layout(width='300'), style=dict(description_width='54px')
)

pretty = dict(
    m11   = "M¹₁₁",  m22   = "M¹₂₂",  m33   = "M¹₃₃",   # ← diagonal of M¹
    M0_11 = "M⁰₁₁",  M0_22 = "M⁰₂₂",  M0_33 = "M⁰₃₃",   # ← diagonal of M⁰
    M0_12 = "M⁰₁₂",  M0_13 = "M⁰₁₃",  M0_23 = "M⁰₂₃"    # ← off-diagonals
)

param_sliders = {k: W.FloatSlider(value=v, description=pretty[k], **slider_cfg)
                 for k,v in pars.items()}

α_slider = W.FloatSlider(
    value=0, min=-5, max=5, step=0.01, continuous_update=True,
    description='α', readout_format='.3f',
    layout=W.Layout(width='690px')
)

info_box = W.HTML(layout=W.Layout(width='300px'))

# -----------------------------------------------------------------
#  fast helpers
# -----------------------------------------------------------------
def recompute_tracks_and_poly():
    global tracks, Λ, poly_sym_tex, poly_disp
    tracks        = eigen_tracks(pars)
    Λ             = asymptotes(pars)
    poly_expr     = sym_poly_expr(pars)
    poly_sym_tex  = sp.latex(poly_expr, order='lex')   # you still have LaTeX
    poly_disp     = fmt_poly(poly_expr)                # …and plain-text
# one-liner rounding helper
round3 = lambda v: f"{v:+.3f}"

# -----------------------------------------------------------------
#  callbacks
# -----------------------------------------------------------------
def on_matrix_slider(change):
    # update parameter dict & recompute heavy stuff
    for k, s in param_sliders.items():
        pars[k] = s.value

    recompute_tracks_and_poly()

    # redraw the 6 static lines
    with fig.batch_update():
        for j in range(3):
            fig.data[j].y      = tracks[:, j]
            fig.data[j+3].y    = α_grid * Λ[j]

    # refresh α-dependent elements so view is consistent
    on_alpha_slider({'new': α_slider.value})

def on_alpha_slider(change):
    a     = change['new']
    idx   = np.clip(np.searchsorted(α_grid, a), 0, nα-1)
    eig   = tracks[idx]
    coeff = np.poly(M0(pars) + a*M1(pars))

    with fig.batch_update():
        # red bar
        fig.data[6].x = [a, a]
        # black cubic
        fig.data[7].y = np.polyval(coeff, λ_plot)
        # dots on x-axis
        for j in range(3):
            fig.data[8+j].x = [eig[j]]
            fig.data[8+j].y = [0]

    # -------- text panel (HTML, no MathJax) ----------------------
    coeff_text = (f"{coeff[0]:+.3g} λ³ {coeff[1]:+.3g} λ² "
                  f"{coeff[2]:+.3g} λ {coeff[3]:+.3g}")

    info_box.value = f"""
<pre style="font-family:monospace">
M₀ =
{np.array2string(M0(pars), precision=2, suppress_small=True)}

M₁ =
{np.array2string(M1(pars), precision=2, suppress_small=True)}

p(λ, α)  = {poly_disp}
p<sub>α</sub>(λ) = {coeff_text}

α = {round3(a)}
ν = [{', '.join(round3(v) for v in Λ)}]
λ = [{', '.join(round3(v) for v in eig)}]
</pre>"""

# -----------------------------------------------------------------
#  wire everything up
# -----------------------------------------------------------------
for s in param_sliders.values():
    s.observe(on_matrix_slider, 'value')   # heavy path (but continuous)

α_slider.observe(on_alpha_slider, 'value') # light path

# initial draw
recompute_tracks_and_poly()
on_alpha_slider({'new': 0.0})

# -----------------------------------------------------------------
#  layout on screen
# -----------------------------------------------------------------
left_panel = W.VBox(list(param_sliders.values()) + [info_box],
                    layout=W.Layout(width='320px'))

display(
    W.VBox([W.HBox([left_panel, fig],
                   layout=W.Layout(align_items='flex-start')),
             α_slider])
)
